# 의미론적 비디오 검색

의미론적 비디오 검색은 키워드나 메타데이터에만 의존하지 않고 의미와 맥락을 기반으로 비디오 콘텐츠를 찾고 검색할 수 있게 하는 고급 기술입니다. 콘텐츠 제작자는 풍부하고 다차원적인 정보가 있는 비디오를 제작합니다. 의미론적 검색 알고리즘은 컴퓨터 비전, 자연어 처리, 그리고 이제는 generative AI와 같은 AI 기술을 사용하여 이 콘텐츠를 분석하고 이해합니다. 이를 통해 사용자는 시스템과 상호작용하여 비디오 내의 특정 개념, 객체 또는 동작을 검색할 수 있습니다.

![Semantic Video Search](./static/images/04-semantic-video-search.png)

의미론적 비디오 검색은 콘텐츠 발견, 사용자 참여 및 전반적인 시청 경험을 극적으로 개선하기 때문에 미디어 및 엔터테인먼트 산업에서 매우 중요합니다. 콘텐츠 과잉의 시대에서 사용자들은 관련 비디오를 찾는 더 효율적인 방법을 요구합니다. 전통적인 검색 방법은 종종 부족하여 사용자의 좌절과 충분히 활용되지 않는 콘텐츠 라이브러리로 이어집니다. 의미론적 검색을 통해 미디어 회사는 비디오 아카이브의 잠재력을 최대한 활용하고, 추천 시스템을 개선하며, 더 개인화된 시청 경험을 만들 수 있습니다.

그러나 효과적인 의미론적 비디오 검색을 구현하는 것에는 상당한 과제가 있습니다. 비디오 데이터의 방대한 양과 복잡성으로 인해 콘텐츠를 정확하게 분석하고 인덱싱하기가 어렵습니다. 비디오 품질, 언어 및 문화적 맥락의 변화는 잘못된 해석으로 이어질 수 있습니다. Generative AI는 의미론적 비디오 검색 기능을 향상시키는 유망한 솔루션을 제공합니다. 대규모 언어 모델과 멀티모달 AI를 활용함으로써, generative AI는 비디오 콘텐츠의 더 미묘하고 맥락을 인식하는 분석을 제공할 수 있습니다. scenes의 상세한 설명을 생성하고, 복잡한 동작과 감정을 식별하며, 심지어 미묘한 문화적 참조도 이해하여 사용자 의도와 비디오 콘텐츠 간의 격차를 해소할 수 있습니다.

이 실습에서는 이전 실습에서 생성된 시각적 및 오디오 메타데이터를 사용하여 멀티모달(MM) 검색 데이터베이스를 구축하기 위한 멀티모달 벡터 데이터베이스를 만들 것입니다. 실습이 끝나면 자연어나 이미지를 사용하여 이 데이터베이스를 쿼리하고 비디오에서 관련 shots를 빠르게 찾을 수 있게 될 것입니다.

# Prerequisites

이 노트북을 실행하려면 노트북 환경을 설정하고 오디오, 시각적, 의미론적 정보를 사용하여 비디오를 세그먼트화한 이전의 모든 기초 노트북을 실행했어야 합니다:
1. [00-prerequisites.ipynb](00-prerequisites.ipyn)
2. [01A-visual-segments-frames-shots-scenes.ipynb](01A-visual-segments-frames-shots-scenes.ipynb) 
3. [01B-audio-segments.ipynb](01B-audio-segments.ipynb) 

### Import python packages

In [ ]:
import json
import boto3
from botocore.exceptions import ClientError
import os
import time
import re
from IPython.display import display, JSON, HTML
import subprocess
from PIL import Image
import base64
from termcolor import colored
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
import random
import datetime

### Retrieve saved values from previous notebooks


In [ ]:
%store -r

In [ ]:
video_path = video['path']
rek_client = boto3.client("rekognition")
bedrock_runtime = boto3.client("bedrock-runtime")
region = sagemaker_resources['region']
oss_host = session['AOSSCollectionEndpoint']

# 아키텍처

이 실습 워크플로우는 SageMaker에서 실행되는 AWS 서비스를 사용합니다. lab01에서 생성된 shot 수준 세그먼테이션 정보(shot 그룹, 오디오 transcription, shot 수준 composite images)를 다양한 AWS AI/GenAI 서비스에 보내 의미론적 비디오 검색 솔루션을 지원하는 임베딩과 메타데이터를 생성합니다. 다음 단계를 완료하게 됩니다:

1. 먼저 shot 수준 프레임을 Amazon Rekognition에 보내 유명인 감지를 수행합니다.
2. 그런 다음 유명인 정보와 shot 수준 composite image를 Bedrock의 Anthropic Claude 3 Sonnet 모델에 보내 shot 수준 캡션을 생성합니다. 그 후 캡션은 역시 Bedrock의 Amazon Titan Text Embedding 모델을 사용하여 텍스트 임베딩으로 변환됩니다.
3. shot 수준 composite image는 또한 이미지 검색 임베딩을 생성하기 위해 Amazon Titan Multi-model(MM) Embedding 모델로 보내집니다.

벡터 데이터베이스로 OpenSearch Serverless에 임베딩과 메타데이터를 인덱싱할 것입니다. 데이터 수집이 완료되면 텍스트와 이미지를 모두 사용하여 해당 데이터베이스를 검색하여 비디오에서 가장 일치하는 shots를 찾을 수 있습니다.

![Flow diagram](./static/images/04-lab-flow-diagram.png)

## 비디오에서 shots의 부분집합을 무작위로 샘플링

더 나은 중단 없는 실습 경험을 위해, 원본 비디오에서 시간순으로 10개의 shots를 무작위로 샘플링할 것입니다. 이 접근 방식은 워크숍 환경의 제한된 용량 때문에 필요합니다. 이렇게 하면 모든 참가자가 제한을 받지 않고 실습을 완료할 수 있도록 하면서도 연습의 무결성을 유지할 수 있습니다. 워크숍 샌드박스 환경에 있지 않다면 shots 수를 늘려도 됩니다.


In [ ]:
shot_mapping = {
    "Netflix_Open_Content_Meridian.mp4": [1, 8, 12, 14, 21, 36, 46, 53, 61, 66],
}

# select the shot mapping
assert video['path'] in shot_mapping, f"****[{video['path']}]*** is not a supported video."

# Assert that the key is in the dictionary
shot_ids = shot_mapping[video['path']]

sampled_shots = []

for shot in video['shots'].shots:

    if shot['id'] in shot_ids:
        sampled_shots.append(shot)
        print(colored(f"Sampled shot id: {shot['id']} ===================\n", "green"))
        display(Image.open(shot['composite_images'][0]['file']))

## Amazon Rekognition을 사용한 유명인 감지
[Amazon Rekognition](https://aws.amazon.com/rekognition/)은 배우, 스포츠인, 온라인 콘텐츠 제작자와 같은 국제적으로 널리 알려진 유명인을 인식하는 데 사용될 수 있습니다. 유명인 인식 API가 제공하는 메타데이터는 콘텐츠에 태그를 지정하고 쉽게 검색할 수 있도록 하는 데 필요한 반복적인 수동 작업을 크게 줄여줍니다. 다음 섹션에서는 이전 단계에서 추출한 shots에서 유명인을 감지하는 데 도움이 되도록 이 기능을 활용할 것입니다.


In [ ]:
def detect_celebrities(shot):
    start_frame_id = shot['start_frame_id']
    end_frame_id = shot['end_frame_id']
    video_asset_dir = shot['video_asset_dir']

    frames = range(start_frame_id, end_frame_id + 1)

    celebrities = set()

    for frame_id in frames:
        try:
            #image_path = f"{video_asset_dir}/frames/frames{frame_id+1:07d}.jpg"
            image_path = f"{video_asset_dir}/frames/frames{frame_id+1:07d}.jpg"
            with open(image_path, 'rb') as image_file:
                image_bytes = image_file.read()      

            # Call Rekognition to detect celebrities
            response = rek_client.recognize_celebrities(
                Image={'Bytes': image_bytes}
            )

            min_confidence = 95.0 # change this value if the accuracy is low.

            for celebrity in response.get('CelebrityFaces', []):
                if celebrity.get('MatchConfidence', 0.0) >= min_confidence:
                    celebrities.add(celebrity['Name'])

        except ClientError as e:
            pass

    public_figures = ', '.join(celebrities)

    shot["public_figure"] = public_figures
    
    return {
            "shot_id": shot['id'],
            "public_figure": public_figures
        }

In [ ]:
print(colored("===== [Celebrities detected in each shot] ======\n", 'green'))
for shot in sampled_shots:
    print(detect_celebrities(shot))

## 오디오 Transcription 처리

자막을 타임스탬프가 있는 문장으로 변환합니다.

In [ ]:
def process_transcript(s):
    subtitle_blocks = re.findall(
        r"(\d+\n(\d{2}:\d{2}:\d{2}.\d{3}) --> (\d{2}:\d{2}:\d{2}.\d{3})\n(.*?)(?=\n\d+\n|\Z))",
        s,
        re.DOTALL,
    )

    sentences = [block[3].replace("\n", " ").strip() for block in subtitle_blocks]
    startTimes = [block[1] for block in subtitle_blocks]
    endTimes = [block[2] for block in subtitle_blocks]

    startTimes_ms = [time_to_ms(time) for time in startTimes]
    endTimes_ms = [time_to_ms(time) for time in endTimes]

    filtered_sentences = []
    filtered_startTimes_ms = []
    filtered_endTimes_ms = []

    startTime_ms = -1
    endTime_ms = -1
    sentence = ""
    for i in range(len(sentences)):
        if startTime_ms == -1:
            startTime_ms = startTimes_ms[i]
        sentence += " " + sentences[i]
        if (
            sentences[i].endswith(".")
            or sentences[i].endswith("?")
            or sentences[i].endswith("!")
            or i == len(sentences) - 1
        ):
            endTime_ms = endTimes_ms[i]
            filtered_sentences.append(sentence.strip())
            filtered_startTimes_ms.append(startTime_ms)
            filtered_endTimes_ms.append(endTime_ms)
            startTime_ms = -1
            endTime_ms = -1
            sentence = ""

    processed_transcript = []
    for i in range(len(filtered_sentences)):
        processed_transcript.append(
            {
                "sentence_startTime": filtered_startTimes_ms[i],
                "sentence_endTime": filtered_endTimes_ms[i],
                "sentence": filtered_sentences[i],
            }
        )

    return processed_transcript

def time_to_ms(time_str):
    h, m, s, ms = re.split(r"[:|.]", time_str)
    return int(h) * 3600000 + int(m) * 60000 + int(s) * 1000 + int(ms)

In [ ]:
with open(video['transcript'].vtt_file, encoding="utf-8") as f:
    transcript = f.read()

processed_transcript = process_transcript(transcript)

print(colored("===== [Complete Sentences W/ Timestamps] ======\n", "green"))
processed_transcript[:5]

## 문장을 shots에 정렬

In [ ]:
def add_shot_transcript(shot_startTime, shot_endTime, transcript):
    relevant_transcript = ""
    for item in transcript:
        if item["sentence_startTime"] >= shot_endTime:
            break
        if item["sentence_endTime"] <= shot_startTime:
            continue
        delta_start = max(item["sentence_startTime"], shot_startTime)
        delta_end = min(item["sentence_endTime"], shot_endTime)
        if delta_end - delta_start >= 500:
            relevant_transcript += item["sentence"] + "; "
    return relevant_transcript

In [ ]:
print(colored("===== [Complete sentence align to every shot] ======\n", "green"))

for shot in sampled_shots:
    relevant_transcript = add_shot_transcript(shot['start_ms'], shot['end_ms'], processed_transcript)
    shot['transcript'] = relevant_transcript
    print({
        'shot_id': shot['id'],
        'transcript': relevant_transcript
    })

## Shot 설명 생성
shot에 속하는 프레임 이미지에서 주요 요소를 추출하기 위해 LLM을 활용합니다.

<div class="alert alert-block alert-info">
⏳ 이 단계는 composite 프레임 이미지에서 주요 요소를 추출하기 위해 Anthropic Claude Sonnet 3.5 모델을 활용하며, 실행하는 데 2분 이상 걸릴 수 있습니다.
</div>

In [ ]:
def get_shot_description(model_id, composite_images, celebrities):

   #  system_prompts = [{"text": """
   #  You are an expert video content analyst specializing in generating rich, contextual metadata for semantic search systems. 
   #  Your task is to analyze video shots presented in a sequence of frame images and provide a detailed but concise description 
   #  of a video shot based on the given frame images. Focus on creating a cohesive narrative of the entire shot rather than 
   #  describing each frame individually.
   #  """}]
    
   #  prompt = """
   #  <celebrities>
   #  {{CELEBRITIES}}
   #  </celebrities>
    
   # Context:
   #  - Each image contains a sequence of consecutive video frames, read from left to right and top to bottom.
   #  - Your goal is to generate metadata that makes the video content easily discoverable through various search queries.
   #  - ALL identified <celebrities> MUST be integrated into descriptions.

   #  STRICT VALIDATION REQUIREMENTS:
   #  1. STOP AND CHECK BEFORE OUTPUTTING:
   #     - Are there any names in the "celebrities" field? 
   #     - If YES, verify these names appear in the description text
   #     - If NO match found, rewrite description to include celebrity names
       
   #  2. REQUIRED FORMAT FOR DESCRIPTIONS WITH CELEBRITIES:
   #     - MUST start with celebrity names and their actions
   #     - Example format: "[Celebrity Name] appears/is shown/portrays..."
   #     - NEVER output generic terms ("a man", "someone") when celebrity identity is known
    
   #  3. AUTOMATIC ERROR CHECKING:
   #     If (celebrities.length > 0):
   #        If (description does not contain ALL celebrity names):
   #           MUST rewrite description
    
   #  Description Template When Celebrities Present:
   #  "[Celebrity Name 1] [action/appearance], [clothing/setting details]. [Additional context]. [Celebrity Name 2 if present] [their action/appearance]..."

   #  REQUIRED PRE-SUBMISSION CHECKS:
   #  □ Are all celebrity names from <celebrities> present in description?
   #  □ Does description start with a celebrity name (not generic terms)?
   #  □ Are all celebrities actively described (not passively mentioned)?
   #  □ Have you avoided generic terms like "a man" or "someone"?
    
   #  INCORRECT (Reject):
   #  "A man in a white shirt and tie is shown..."
   #  (When celebrities field contains "Kevin Kilner")
    
   #  CORRECT (Accept):
   #  "Kevin Kilner appears in a white shirt and tie..."
    
   #  Input Description:
   #  <input_description>
   #  - Sequence of images representing video frames
   #  - List of known celebrities (if applicable)
   #  </input_description>

   #  Step-by-step Instructions:
   #  <instructions>
   #  1. Analyze the visual content:
   #     a. First priority: Identify any celebrities or notable individuals
   #     b. Check for dark/empty frames:
   #        - If frames are black or empty, use specialized template
   #        - Set appropriate technical descriptors
   #        - Mark confidence scores as 100 for verified empty content
   #        - Use "None" or "Undefined" for inapplicable categories
   #     c. If celebrities identified, prepare description using required template
   #     d. Identify key objects, actions, and settings in the scene
   #     e. Detect any text or graphics visible in the frames
   #     f. Recognize brands, logos, or products
    
   #  2. Determine temporal aspects:
   #     a. Identify any scene transitions or significant changes in the sequence
   #     b. Note any recurring elements across multiple frames
    
   #  3. Synthesize a detailed description:
   #     a. REQUIRED: If celebrities present, use template format
   #     b. MUST start with celebrity identification and actions
   #     c. Integrate setting, atmosphere, and context
   #     d. Include all identified celebrities in natural narrative flow
   #     e. Run pre-submission checks before finalizing
    
   #  4. Final Validation:
   #     a. Run through pre-submission checklist
   #     b. Verify celebrity integration in description (if applicable)
   #     c. Confirm no generic terms used for identified people
   #     d. For dark frames, verify all technical descriptors are accurate
    
   #  5. Special Cases Handling:
   #      a. For dark/empty frames:
   #         - Use technical description template
   #         - Set appropriate null values
   #         - Mark relevant technical indicators
   #         - Note possible transition purpose
   #      b. For partially visible content:
   #         - Note visibility issues
   #         - Describe what can be confidently identified
   #         - Adjust confidence scores accordingly
    
   #  6. Final Output Preparation:
   #      a. Skip the preamble; go straight into the description
   #      b. Check for proper formatting and syntax
   #  """.replace("{{CELEBRITIES}}", celebrities)

    system_prompts = [{"text": """
    당신은 의미론적 검색 시스템을 위한 풍부한 맥락적 메타데이터를 생성하는 전문 비디오 콘텐츠 분석가입니다. 
    당신의 임무는 프레임 이미지 시퀀스로 제시된 비디오 shots를 분석하고 주어진 프레임 이미지를 기반으로 
    비디오 shot에 대한 상세하지만 간결한 설명을 제공하는 것입니다. 각 프레임을 개별적으로 
    설명하기보다는 전체 shot의 일관된 내러티브를 만드는 데 집중하세요.
    """}]

    prompt = """
    <celebrities>
    {{CELEBRITIES}}
    </celebrities>
    
   맥락:
    - 각 이미지는 왼쪽에서 오른쪽, 위에서 아래로 읽는 연속적인 비디오 프레임의 시퀀스를 포함합니다.
    - 당신의 목표는 다양한 검색 쿼리를 통해 비디오 콘텐츠를 쉽게 발견할 수 있게 하는 메타데이터를 생성하는 것입니다.
    - 식별된 모든 <celebrities>는 반드시 설명에 통합되어야 합니다.

    엄격한 검증 요구사항:
    1. 출력 전 중지 및 확인:
       - "celebrities" 필드에 이름이 있습니까? 
       - 있다면, 이 이름들이 설명 텍스트에 나타나는지 확인
       - 일치하는 것이 없다면, 유명인 이름을 포함하도록 설명 다시 작성
       
    2. 유명인이 있는 설명에 대한 필수 형식:
       - 반드시 유명인 이름과 그들의 행동으로 시작해야 함
       - 예시 형식: "[Celebrity Name] 등장/보여짐/묘사..."
       - 유명인 신원을 알 때는 절대 일반적인 용어("한 남자", "누군가")를 출력하지 않음
    
    3. 자동 오류 검사:
       If (celebrities.length > 0):
          If (설명이 모든 유명인 이름을 포함하지 않음):
             반드시 설명 다시 작성
    
    유명인이 있을 때의 설명 템플릿:
    "[Celebrity Name 1] [행동/등장], [의상/배경 세부사항]. [추가 맥락]. [Celebrity Name 2가 있다면] [그들의 행동/등장]..."

    필수 제출 전 확인사항:
    □ <celebrities>의 모든 유명인 이름이 설명에 있습니까?
    □ 설명이 유명인 이름으로 시작합니까(일반적인 용어가 아님)?
    □ 모든 유명인이 능동적으로 설명되었습니까(수동적으로 언급된 것이 아님)?
    □ "한 남자" 또는 "누군가"와 같은 일반적인 용어를 피했습니까?
    
    잘못됨 (거부):
    "흰 셔츠와 넥타이를 입은 한 남자가 보입니다..."
    (celebrities 필드에 "Kevin Kilner"가 있을 때)
    
    올바름 (수락):
    "Kevin Kilner가 흰 셔츠와 넥타이를 입고 등장합니다..."
    
    입력 설명:
    <input_description>
    - 비디오 프레임을 나타내는 이미지 시퀀스
    - 알려진 유명인 목록 (해당되는 경우)
    </input_description>

    단계별 지침:
    <instructions>
    1. 시각적 콘텐츠 분석:
       a. 첫 번째 우선순위: 유명인 또는 주목할 만한 개인 식별
       b. 어둡거나 빈 프레임 확인:
          - 프레임이 검거나 비어 있다면, 특수 템플릿 사용
          - 적절한 기술적 설명자 설정
          - 확인된 빈 콘텐츠에 대해 신뢰도 점수를 100으로 설정
          - 해당되지 않는 카테고리에 대해 "None" 또는 "Undefined" 사용
       c. 유명인이 식별되면, 필수 템플릿을 사용하여 설명 준비
       d. 장면의 주요 객체, 행동 및 배경 식별
       e. 프레임에서 보이는 텍스트나 그래픽 감지
       f. 브랜드, 로고 또는 제품 인식
    
    2. 시간적 측면 결정:
       a. 장면 전환이나 시퀀스의 중요한 변화 식별
       b. 여러 프레임에 걸쳐 반복되는 요소 기록
    
    3. 상세한 설명 종합:
       a. 필수: 유명인이 있다면, 템플릿 형식 사용
       b. 반드시 유명인 식별과 행동으로 시작
       c. 배경, 분위기 및 맥락 통합
       d. 자연스러운 내러티브 흐름에 모든 식별된 유명인 포함
       e. 최종화하기 전에 제출 전 확인 실행
    
    4. 최종 검증:
       a. 제출 전 체크리스트 실행
       b. 설명에서 유명인 통합 확인 (해당되는 경우)
       c. 식별된 사람들에 대해 일반적인 용어를 사용하지 않았는지 확인
       d. 어두운 프레임의 경우, 모든 기술적 설명자가 정확한지 확인
    
    5. 특수 사례 처리:
        a. 어둡거나 빈 프레임의 경우:
           - 기술적 설명 템플릿 사용
           - 적절한 null 값 설정
           - 관련 기술적 지표 표시
           - 가능한 전환 목적 기록
        b. 부분적으로 보이는 콘텐츠의 경우:
           - 가시성 문제 기록
           - 확실히 식별할 수 있는 것 설명
           - 그에 따라 신뢰도 점수 조정
    
    6. 최종 출력 준비:
        a. 서문 건너뛰고 바로 설명으로 들어가기
        b. 적절한 형식과 구문 확인
    """.replace("{{CELEBRITIES}}", celebrities)


    message = {
        "role":"user",
        "content":[]
    }
                         
    for composite in composite_images:

        with open(composite['file'], "rb") as image_file:
            image_string = image_file.read()

        message["content"].append({
            "image":{
                "format": "jpeg",
                "source":{
                    "bytes": image_string
                }
            }
                
        })

    message["content"].append({
        "text": prompt
    })
    
    # Base inference parameters to use.
    inference_config = {"temperature": .1}
    
    # Additional inference parameters to use.
    additional_model_fields = {"top_k": 200}
    
    response = bedrock_runtime.converse(
        modelId=model_id,
        messages=[message],
        system=system_prompts,
        inferenceConfig=inference_config,
        additionalModelRequestFields=additional_model_fields
    )
    output_message = response['output']['message']
    
    return output_message["content"][0]["text"]

shot 설명을 생성하기 위해 코드를 실행합니다.

In [ ]:
%%time
# model_id = 'anthropic.claude-3-5-sonnet-20240620-v1:0'
# model_id = "anthropic.claude-3-haiku-20240307-v1:0"
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

for shot in sampled_shots:
    description = get_shot_description(
        model_id = model_id, 
        composite_images = shot['composite_images'], 
        celebrities = shot['public_figure']
    )
    shot['shot_description'] = description

print(colored("===== [Example caption] ======\n", "green"))

example = random.choice(sampled_shots)
example['shot_description']


## Shots에 대한 임베딩 생성

이 함수는 제공된 model_id를 기반으로 텍스트 또는 멀티모달 임베딩을 생성합니다

In [ ]:
def get_embedding(model_id, input_data):
    accept = "application/json"
    content_type = "application/json"

    if 'text' in model_id:
        body = json.dumps({
            "inputText": input_data,
            "dimensions": 1024,
            "normalize": True
        })
    elif 'image' in model_id:
        # Read image from file and encode it as base64 string.
        with open(input_data, "rb") as image_file:
            input_image = base64.b64encode(image_file.read()).decode('utf8')
        
        body = json.dumps({
            "inputImage": input_image,
            "embeddingConfig": {
                "outputEmbeddingLength": 1024
            }
        })
    else:
        raise ValueError("Invalid embedding_type. Choose 'text' or 'image'.")

    response = bedrock_runtime.invoke_model(
        body=body,
        modelId=model_id,
        accept=accept,
        contentType=content_type,
    )
    response_body = json.loads(response["body"].read())
    embedding = response_body.get("embedding")

    return embedding

## OpenSearch Serverless 벡터 인덱스 구축

OpenSearch Serverless (OSS)는 AWS(Amazon Web Services)에서 제공하는 완전 관리형, 온디맨드 검색 및 분석 서비스입니다. 인프라 관리 없이 OpenSearch 클러스터를 배포, 운영 및 확장할 수 있습니다.

OpenSearch의 인덱스는 유사한 특성을 공유하는 문서의 모음입니다. 이 경우, 우리는 벡터 임베딩을 효율적으로 저장하고 검색하도록 설계된 벡터 인덱스에 초점을 맞추고 있습니다.

### 인덱스 구성은 다음과 같습니다

인덱스는 다음 속성들을 포함합니다:
- `video_path`: 비디오 파일 경로 (텍스트 필드)
- `shot_id`: 각 shot의 고유 식별자 (텍스트 필드)
- `shot_startTime`: shot의 시작 시간 (텍스트 필드)
- `shot_endTime`: shot의 종료 시간 (텍스트 필드)
- `shot_description`: shot의 설명 (텍스트 필드)
- `shot_celebrities`: shot에서 식별된 유명인 (텍스트 필드)
- `shot_transcript`: shot의 오디오 Transcript (텍스트 필드)

이것들은 각 검색 쿼리에 대한 shots를 검색하고 결과를 필터링하는 데 사용할 수 있는 메타데이터 필드입니다.

- `shot_image_vector`: shot 이미지의 벡터 표현
- `shot_desc_vector`: shot 설명의 벡터 표현
- `transcript_vector`: transcript의 벡터 표현

`shot_image_vector`, `transcript_vector` 및 `shot_desc_vector`는 `knn_vector` 필드로 구성됩니다. 이 두 필드를 사용하여 텍스트 쿼리나 입력 이미지에 해당하는 가장 일치하는 카메라 shot을 찾기 위해 벡터 유사도 검색을 수행할 것입니다.

In [ ]:
# Establish client connection OSS
def get_opensearch_client(host, region):
    host = host.split("://")[1] if "://" in host else host
    credentials = boto3.Session().get_credentials()
    auth = AWSV4SignerAuth(credentials, region, "aoss")

    oss_client = OpenSearch(
        hosts=[{"host": host, "port": 443}],
        http_auth=auth,
        use_ssl=True,
        verify_certs=True,
        connection_class=RequestsHttpConnection,
        pool_maxsize=20,
    )

    return oss_client


# Create OpenSearch Severless Index
def create_opensearch_index(oss_client, index_name, len_embedding=1024):

    exist = oss_client.indices.exists(index_name)
    if not exist:
        print("Creating index")
        index_body = {
            "mappings": {
                "properties": {
                    "video_path": {"type": "text"},
                    "shot_id": {"type": "text"},
                    "shot_startTime": {"type": "text"},
                    "shot_endTime": {"type": "text"},
                    "shot_description": {"type": "text"},
                    "shot_celebrities": {"type": "text"},
                    "shot_transcript": {"type": "text"},
                    "shot_image_vector": {
                        "type": "knn_vector",
                        "dimension": len_embedding,
                        "method": {
                            "engine": "nmslib",
                            "space_type": "cosinesimil",
                            "name": "hnsw",
                            "parameters": {"ef_construction": 512, "m": 16},
                        },
                    },
                    "shot_desc_vector": {
                        "type": "knn_vector",
                        "dimension": len_embedding,
                        "method": {
                            "engine": "nmslib",
                            "space_type": "cosinesimil",
                            "name": "hnsw",
                            "parameters": {"ef_construction": 512, "m": 16},
                        },
                    },
                    "transcript_vector": {
                        "type": "knn_vector",
                        "dimension": len_embedding,
                        "method": {
                            "engine": "nmslib",
                            "space_type": "cosinesimil",
                            "name": "hnsw",
                            "parameters": {"ef_construction": 512, "m": 16},
                        },
                    },
                }
            },
            "settings": {
                "index": {
                    "number_of_shards": 2,
                    "knn.algo_param": {"ef_search": 512},
                    "knn": True,
                }
            },
        }
        response = oss_client.indices.create(index_name, body=index_body)

        print(response)

In [ ]:
index_name = "video_search_index"

oss_client = get_opensearch_client(oss_host, region)
create_opensearch_index(oss_client, index_name)

OpenSearch Serverless (OSS)에 데이터를 수집하기 위한 수집 페이로드를 생성하기 위해 각 shot을 반복합니다

In [ ]:
for shot in sampled_shots:

    # generate text embedding from description
    shot_desc_vector = get_embedding(
        model_id='amazon.titan-embed-text-v2:0',
        input_data=shot['shot_description']
    )

    # generate mm embedding from composite frames
    shot_image_vector = get_embedding(
        model_id='amazon.titan-embed-image-v1',
        input_data=shot['composite_images'][0]['file']
    )

    index_obj = {
                "video_path": video_path,
                "shot_id": shot['id'],
                "shot_startTime": shot['start_ms'],
                "shot_endTime": shot['end_ms'],
                "shot_description": shot['shot_description'],
                "shot_celebrities": shot['public_figure'],
                "shot_transcript": shot['transcript'],
                "shot_desc_vector": shot_desc_vector,
                "shot_image_vector": shot_image_vector,
            }

    # generate text embedding from transcript
    if shot['transcript']:
        
        transcript_vector = get_embedding(
            model_id='amazon.titan-embed-text-v2:0',
            input_data=shot['transcript']
        )
        
        index_obj["transcript_vector"] = transcript_vector
        
    #build the payload to index in OSS
    payload = json.dumps(index_obj)
    response = oss_client.index(
                    index=index_name,
                    body=payload,
                    params={"timeout": 60},
                )

## 비디오 의미론적 검색 수행

OpenSearch에 삽입된 데이터가 검색될 준비가 되었는지 확인하기 위해 기다립니다.


In [ ]:
print("Waiting for the recent inserted data to be searchable in OpenSearch...")

while True:
    try:
        result = oss_client.search(index=index_name, body={"query": {"match_all": {}}})
        if result['hits']['total']['value'] >= len(sampled_shots):
            print("\nData is now available for search!")
            break
    except Exception as e:
        print(".", end="", flush=True)
        time.sleep(10)

자연어 쿼리를 사용한 검색을 시연합니다. 기본적으로 Question Bank에서 질문이 무작위로 샘플링됩니다. 그런 다음 검색은 OSS 인덱스의 시각적(`shot_desc_vector`) 및 오디오(`transcript_vector`) 데이터를 모두 결합합니다. 이 프로세스는 이러한 유형의 비디오에서 최적화된 결과를 위해 시각적(75%)을 오디오 전사(25%)보다 우선시하도록 콘텐츠 가중치를 적용합니다. 이 결합된 검색 예시는 이러한 매개변수를 다른 사용자 검색 의도에 더 잘 맞도록 조정하여 전반적인 검색 관련성과 사용자 만족도를 향상시킬 수 있는 방법을 보여줍니다.

In [ ]:
question_bank = {
    "Netflix_Open_Content_Meridian.mp4": [
            "Scott driving a car",
            "Elyse staring through the rear view mirror",
            "Kevin opening a car door",
            "Lightning strike from the sky"
        ]    
}
# check if questions are available for the video
assert video['path'] in question_bank, f"****[{video['path']}]*** is not a supported video."

# Sample a question from the question bank. You can change to use your own.
user_query = random.choice(question_bank[video['path']])

print("Sampled query: ", colored(user_query, "green"))

query_embedding = get_embedding('amazon.titan-embed-text-v2:0', user_query)

In [ ]:
aoss_query = {
        "size": 10,
        "query": {
            "bool": {
                "should": [
                    {
                        "script_score": {
                            "query": {"match_all": {}},
                            "script": {
                                "lang": "knn",
                                "source": "knn_score",
                                "params": {
                                    "field": "shot_desc_vector",
                                    "query_value": query_embedding,
                                    "space_type": "cosinesimil",
                                },
                            },
                            "boost": 3.0
                        }
                    },
                    {
                        "script_score": {
                            "query": {"match_all": {}},
                            "script": {
                                "lang": "knn",
                                "source": "knn_score",
                                "params": {
                                    "field": "transcript_vector",
                                    "query_value": query_embedding,
                                    "space_type": "cosinesimil",
                                },
                            },
                            "boost": 1.0
                        }
                    }
                ],
                "minimum_should_match": 1
            }
        },
        "_source": [
            "video_path",
            "shot_id",
            "shot_startTime",
            "shot_endTime",
            "shot_description",
            "shot_celebrities",
            "shot_transcript",
        ],
    }

In [ ]:
response = oss_client.search(body=aoss_query, index=index_name)
hits = response["hits"]["hits"]

responses = []
for hit in hits:
    if hit["_score"] >= 0:  # Set score threshold
        responses.append(
            {
                "video_path": hit["_source"]["video_path"],
                "shot_id": hit["_source"]["shot_id"],
                "shot_startTime": hit["_source"]["shot_startTime"],
                "shot_endTime": hit["_source"]["shot_endTime"],
                "shot_description": hit["_source"]["shot_description"],
                "shot_celebrities": hit["_source"]["shot_celebrities"],
                "shot_transcript": hit["_source"]["shot_transcript"],
                "score": hit["_score"],
            }
        )

상위 `x`개의 검색 결과를 표시합니다. 아래의 헬퍼 함수는 shots를 독립적으로 그리고 원본 비디오의 일부로 렌더링합니다.

In [ ]:
def render_with_original_video(top_hit):
    
    video_path = top_hit['video_path']
    video_start = top_hit['shot_startTime']/1000
    
    display(HTML(f"""
    <video alt="test" controls id="{top_hit['shot_id']}" width="100" >
      <source src="{video_path}">
    </video>
    
    <script>
    video = document.getElementById("{top_hit['shot_id']}")
    video.currentTime = {video_start};
    </script>
    """))
    
    
def display_shot_segment_results(response, top_results=2):

    css_style = """
    <style>
        .video-container {
            display: flex;
            justify-content: space-around;
            flex-wrap: wrap;
        }
        .video {
            flex: 1;
            min-width: 200px;
            margin: 10px;
        }
        video {
            width: 100%;
            height: auto;
        }
    </style>
    """

    html_content = "<div class='video-container'>\n"
    
    for idx in range(top_results):
        # convert format of timestamps
        video_start = responses[idx]['shot_startTime']/1000
        video_end = responses[idx]['shot_endTime']/1000
    
        converted_start = str(datetime.timedelta(seconds = video_start))
        converted_end = str(datetime.timedelta(seconds = video_end))
        output_file = f"shot-{responses[idx]['shot_id']}.mp4"
        _ = subprocess.run(
            [
                "/usr/bin/ffmpeg",
                "-ss",
                converted_start,
                "-to",
                converted_end,
                "-i",
                responses[idx]['video_path'], # path to video
                "-c",
                "copy",
                output_file,
            ],
            stderr=subprocess.PIPE
        )
        html_content += f"""
            <div class="video">
                <h5>Shot Id: {responses[idx]['shot_id']}, Time Range: {video_start} ms - {video_end} ms</p>
                <video controls>
                    <source src="{output_file}" type="video/mp4">
                    Your browser does not support the video tag.
                </video>
            </div>
        """
    # render the shots
    html_content += "</div>"
    
    display(HTML(css_style + html_content))

In [ ]:
print(colored("====== [TOP results] =======", 'green'))
display_shot_segment_results(responses, top_results=3)

print(colored("\n====== [Display top hit as part Of original video] =======\n", 'green'))

top_hit = responses[0]

video_start = top_hit['shot_startTime']/1000
video_end = top_hit['shot_endTime']/1000

print(f"Shot Id: {top_hit['shot_id']}, Time Range: {video_start} ms - {video_end} ms")
render_with_original_video(top_hit)

### 멀티모달 비디오 검색

콘텐츠 분석 및 비디오 편집 워크플로우에서, 비디오 제작자는 핵심 순간을 완벽하게 포착하는 특정 프레임이나 이미지를 발견할 수 있지만 수 시간의 원본 영상 내에서 그 위치를 찾아야 할 수 있습니다. 이 Generative AI 기술을 사용한 멀티모달 비디오 검색을 통해, 제작자는 프레임이나 이미지를 입력하여 프레임이 발생하는 정확한 타임스탬프를 빠르게 찾을 수 있습니다.

다음 검색 예시에서는 사용 가능한 shots에서 프레임을 무작위로 샘플링한 다음, 프레임 이미지를 사용하여 비디오에서 shot을 식별할 것입니다.


In [ ]:
def random_sample_image(shots):
    shot = random.choice(shots) if shots else None
    frame_locations = shot['composite_images'][0]['layout']
    frame_info = random.choice(frame_locations) if frame_locations else None
    return frame_info[0]

random_frame = random_sample_image(sampled_shots)
image = Image.open(random_frame)
image.show()

In [ ]:
image_embedding = get_embedding('amazon.titan-embed-image-v1', random_frame)

In [ ]:
aoss_query = {
        "size": 10,
        "query": {
            "script_score": {
                "query": {"bool": {"should": []}},
                "script": {
                    "lang": "knn",
                    "source": "knn_score",
                    "params": {
                        "field": "shot_image_vector",
                        "query_value": image_embedding,
                        "space_type": "cosinesimil",
                    },
                },
            }
        },
        "_source": [
            "video_path",
            "shot_id",
            "shot_startTime",
            "shot_endTime",
            "shot_description",
            "shot_celebrities",
            "shot_transcript",
        ],
    }

In [ ]:
response = oss_client.search(body=aoss_query, index=index_name)
hits = response["hits"]["hits"]

responses = []
for hit in hits:
    if hit["_score"] >= 0:  # Set score threshold
        responses.append(
            {
                "video_path": hit["_source"]["video_path"],
                "shot_id": hit["_source"]["shot_id"],
                "shot_startTime": hit["_source"]["shot_startTime"],
                "shot_endTime": hit["_source"]["shot_endTime"],
                "shot_description": hit["_source"]["shot_description"],
                "shot_celebrities": hit["_source"]["shot_celebrities"],
                "shot_transcript": hit["_source"]["shot_transcript"],
                "score": hit["_score"],
            }
        )

In [ ]:
print(colored("====== [TOP results] =======", 'green'))
display_shot_segment_results(response, top_results=3)

print(colored("\n====== [Display top hit as part Of original video] =======\n", 'green'))

top_hit = responses[0]

video_start = top_hit['shot_startTime']/1000
video_end = top_hit['shot_endTime']/1000

print(f"Shot Id: {top_hit['shot_id']}, Time Range: {video_start} ms - {video_end} ms")
render_with_original_video(top_hit)

## Clean Up
방금 생성한 벡터 인덱스를 제거하려면 아래 코드의 주석을 해제하세요.

In [ ]:
# try:
#     response = oss_client.indices.delete(index=index_name)
#     print(f"Index '{index_name}' deleted successfully")
# except Exception as e:
#     print(f"Error deleting index '{index_name}': {str(e)}")

# 다음은 무엇인가요?

다른 사용 사례를 시도해보거나, 완료했다면 [Additional Resources](09-resources.ipynb) 실습으로 계속 진행할 수 있습니다.